## **Projet 1 : Fine-tuning d’un Modèle LLAMA 3**  

### 🎯 **Objectif**  
L’objectif de cet exercice est de **reprendre toutes les étapes** du notebook pour fine-tuner un modèle **LLAMA 3**, afin qu’il puisse répondre à la question :  
**"Quelles sont les champs d'activité de l'INRAE ?"**  

Vous devrez :   
✅ **Télécharger et préparer le modèle LLAMA 3**  
✅ **Créer et pré-traiter un dataset spécifique**  
✅ **Effectuer le fine-tuning avec LoRA**  
✅ **Sauvegarder le modèle entraîné**  
✅ **Convertir le modèle en format GGUF pour Ollama**  
✅ **Tester le modèle en lui posant la question cible**  

In [ ]:
# Import du modèle depuis hugging face




In [ ]:
# Traitement du jeu de données




In [ ]:
# Entrainement du modèle



In [ ]:
# Sauvegarde du modèle




In [ ]:
# Convertion du modèle avec CCP et Ollama



